# Create a SNANA ".fitres" file from selected SNe

The list then will be used as input in "SNANA_SALT2_DistanceMu_v0_1.ipynb" to compute the 
distance moduli, but over all, the RESIDUAL distance moduli from this particular subsample.

## User

In [ ]:
import numpy as np

# Directory where are located the list "SNe_inGPHD_AndySample_Repeated_Notes_.txt" 
# with the names of the SNe that I want to create the ".fitres" file from.
DirLists = '/Users/arturo/Dropbox/Research/SoftwareResearch/Snana/Odyssey/home_snana/lowz/'

ListSNeToBeUsed = 'SNe_inGPHD_AndySample_Repeated_Notes_v3.txt'
ListSNe = np.genfromtxt(DirLists+ListSNeToBeUsed, dtype=['S30', 'S43', 'S40'])

DirSNANAFittedFolders = DirLists

#######################################################

#    CUTOFFS

# Apply additional cutoffs to the sample?:
ApplyCutoffs = True

if ApplyCutoffs == True:
    c_limits = -0.3, 0.3   # color
    x1_limits = -3, 3   # light-curve parameter
    x1ERR_limits = 1    # error in the light-curve parameter
    PKMJDERR_limits = 2  # error in the determination of MJD at T_Bmax
    FITPROB_limits = 0.001   # probability of the fit

#######################################################
    
print '%s SNe in the list (regardless the cutoffs).'%len(ListSNe)
# 50 SNe in the list.

In [ ]:
5+4

-----

## Useful functions

#### Function to identify string or number

In [ ]:
# Function to identify if a string is an integer number or a letter.
# This will be used in the dictionary construction to properly read some SN names.

def is_number(s):
    try:
        int(s)
        return True
    except ValueError:
        return False

# Tests
print is_number('5'), is_number('e')
# True False

#### Get the name of this ipython notebook
To print it in the output text files as reference

In [ ]:
%%javascript
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "NotebookName = " + "'"+thename+".ipynb"+"'";
kernel.execute(command);

In [ ]:
print '#', (NotebookName)
# Update_zcmb_in_SNANA_datafiles_v1_0.ipynb

In [ ]:
# Get the current date and time
import datetime 

# Read the time and date now
now = datetime.datetime.now()

---------

## Automatic

In [ ]:
if ApplyCutoffs == True:
    DirSaveOutput = DirLists+'GaussianProcessSubsample/cutoffs_yes/'
elif ApplyCutoffs == False:
    DirSaveOutput = DirLists+'GaussianProcessSubsample/cutoffs_no/'
    
#- Force the creation of the directory to save the outputs.
#- "If the subdirectory does not exist then create it"
import os # To use command line like instructions
if not os.path.exists(DirSaveOutput): os.makedirs(DirSaveOutput)

#### Main loop

In [ ]:
file_1 = open(DirSaveOutput+'GP_subsample_.fitres','w')

# Write down the SNANA headers:

now = datetime.datetime.now() # Read the time and date right now
text_timenow = now.strftime("%Y-%m-%d (yyyy/mm/dd); %H:%M hrs.")
text_line = '#'+'-'*50 + '\n'


# These two lines MUST be at the first ones on the text file in order to be 
# easily readed by "SNANA_SALT2_DistanceMu_v1_1.ipynb"
file_1.write('NVAR:  24 \n')
file_1.write('VARNAMES: CID       z        zERR     x0           x0ERR        c      \
cERR     x1      x1ERR   PKMJD   PKMJDERR mB      mBERR    COVx0x1     COVx0c      \
COVx1c     CHI2   NDOF  FITPROB      SNRMAX1    SNRMAX2    SNRMAX3 IDSURVEY TYPE    \n')
file_1.write(text_line)


file_1.write('# SNANA-like "fitres" file created from selected SNe from the list: \n')
file_1.write('# %s \n'%ListSNeToBeUsed)

file_1.write('# Data table created by: Arturo Avelino \n')
file_1.write('# On date: %s \n'%text_timenow)
file_1.write('# Script used: %s \n'%NotebookName)
file_1.write(text_line)

if ApplyCutoffs == True:
    file_1.write("# Cutoff applied: %s < c < %s | %s < x1 < %s | x1ERR < %s,  \n"%
                 (c_limits[0], c_limits[1], x1_limits[0], x1_limits[1], x1ERR_limits) )
    file_1.write("# PKMJDERR < %s | FITPROB > %s. \n"%(PKMJDERR_limits, FITPROB_limits))
    file_1.write(text_line)


#---------------------------------------------------------

countSN = 0
for i in range(len(ListSNe)):
    name          = ListSNe['f0'][i]
    snSnanaFolder = ListSNe['f2'][i]

    # Read correctly the name of the SNe.
    if   name[7] == '_': snName = name[2:7] # To read correctly, e.g., "sn2011B"
    elif name[7] != '_':
        if is_number(name[7]): snName = name[2:15] # To read correctly, e.g., "snf20080514-002"
        else: snName = name[2:8]  # To read correctly, e.g., "sn1998bu" 
    
    Dir_int1 = DirSNANAFittedFolders+snSnanaFolder+'/'
    
    fitresFile = np.genfromtxt(Dir_int1+snSnanaFolder+'.fitres', skip_header=2,
                                dtype=['S3', 'S15',
                                      float, float, float, float, float, float, float,
                                      float, float, float, float, float, float, float, float,
                                      float, float, float, float, float, float, float, float]) 

    # Find the index where the SN is located in the fitres file.
    index_int = np.where(fitresFile['f1'] == snName)[0][0]
            
    c_par  = fitresFile['f6'][index_int]
    x1_par = fitresFile['f8'][index_int]
    x1ERR_par = fitresFile['f9'][index_int]
    PKMJDERR_par = fitresFile['f11'][index_int]
    FITPROB_par  = fitresFile['f19'][index_int]

    if ApplyCutoffs == True:
        # SNe that pass the cutoffs:
        if (c_par > c_limits[0] and c_par < c_limits[1] and
            x1_par > x1_limits[0] and x1_par < x1_limits[1] and
            x1ERR_par < x1ERR_limits and PKMJDERR_par < PKMJDERR_limits and
            FITPROB_par > FITPROB_limits):
            
            snName_print = 'SN: %-13s'%fitresFile['f1'][index_int]
            countSN = countSN + 1

        else: 
            snName_print = '## SN: %-13s'%fitresFile['f1'][index_int]
        
    else: 
        snName_print = 'SN: %-13s'%fitresFile['f1'][index_int]
        countSN = countSN + 1

    # Write down a line in the text list with all the fitres information for a given SN. 
    file_1.write('%s  %.5f  %.5f  %.5e  %.3e  %7.4f  %.4f  %7.4f  %.4f  %.3f  %.2f \
%.4f  %.4f  %10.3e  %10.3e  %10.3e  %5.1f  %4.0f  %.3e  %9.3f  %9.3f  %9.3f  %2.0f     %2.0f\n'%(  
        snName_print, fitresFile['f2'][index_int], fitresFile['f3'][index_int],
        fitresFile['f4'][index_int], fitresFile['f5'][index_int], fitresFile['f6'][index_int],
        fitresFile['f7'][index_int], fitresFile['f8'][index_int], fitresFile['f9'][index_int],
        fitresFile['f10'][index_int], fitresFile['f11'][index_int],
        fitresFile['f12'][index_int], fitresFile['f13'][index_int],
        fitresFile['f14'][index_int], fitresFile['f15'][index_int], fitresFile['f16'][index_int],
        fitresFile['f17'][index_int], fitresFile['f18'][index_int], fitresFile['f19'][index_int],
        fitresFile['f20'][index_int], fitresFile['f21'][index_int], fitresFile['f22'][index_int],
        fitresFile['f23'][index_int], fitresFile['f24'][index_int]  ))

    print snName_print
    

text_10 = '# %s SNe Ia passed the cutoffs (if applied). \n'%countSN
text_11 = "# The commented SNe (##) didn't pass the cutoffs."
file_1.write(text_10); file_1.write(text_11)
file_1.close()

print text_10
print text_11